In [1]:
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# Qwen3-VL 属于视觉语言多模态模型，其配置类 Qwen3VLConfig 并不在 AutoModelForCausalLM 支持的纯文本模型列表中

# 模型路径（替换为你下载的本地路径）
MODEL_PATH = r"E:/models/Qwen3-VL-2B-Instruct"

In [2]:
# 加载处理器（Qwen3-VL 用 AutoProcessor，集成了图像和文本处理）
processor = AutoProcessor.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    use_fast=False  # Qwen 系列推荐关闭 fast tokenizer
)

# 加载模型
model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype="auto", 
    device_map="auto"
)

model.eval()  # 推理模式

Qwen3VLForConditionalGeneration(
  (model): Qwen3VLModel(
    (visual): Qwen3VLVisionModel(
      (patch_embed): Qwen3VLVisionPatchEmbed(
        (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (pos_embed): Embedding(2304, 1024)
      (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-23): 24 x Qwen3VLVisionBlock(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Qwen3VLVisionAttention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (mlp): Qwen3VLVisionMLP(
            (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (act_fn): GELUTanh()
          )
        )
      )
 

In [3]:
print("模型结构概述：")
print(model)

模型结构概述：
Qwen3VLForConditionalGeneration(
  (model): Qwen3VLModel(
    (visual): Qwen3VLVisionModel(
      (patch_embed): Qwen3VLVisionPatchEmbed(
        (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (pos_embed): Embedding(2304, 1024)
      (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-23): 24 x Qwen3VLVisionBlock(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Qwen3VLVisionAttention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (mlp): Qwen3VLVisionMLP(
            (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (act_fn): GELUTanh()
          )
        )
 

In [4]:
# 打印模型参数总量
total_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {total_params / 1e9:.2f}B")

模型总参数量: 2.13B


In [5]:
# 打印配置（查看超参数，如隐藏层大小、层数等）
print("模型配置：")
print(model.config)

模型配置：
Qwen3VLConfig {
  "architectures": [
    "Qwen3VLForConditionalGeneration"
  ],
  "image_token_id": 151655,
  "model_type": "qwen3_vl",
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "intermediate_size": 6144,
    "max_position_embeddings": 262144,
    "model_type": "qwen3_vl_text",
    "num_attention_heads": 16,
    "num_hidden_layers": 28,
    "num_key_value_heads": 8,
    "rms_norm_eps": 1e-06,
    "rope_scaling": {
      "mrope_interleaved": true,
      "mrope_section": [
        24,
        20,
        20
      ],
      "rope_type": "default"
    },
    "rope_theta": 5000000,
    "tie_word_embeddings": true,
    "use_cache": true,
    "vocab_size": 151936
  },
  "tie_word_embeddings": true,
  "transformers_version": "4.57.3",
  "video_token_id": 151656,
  "vis

In [7]:
# 1. 基础用法：纯文本交互（无图像，仅文本对话）
def pure_text_interaction(prompt: str, max_new_tokens: int = 512) -> str:
    """
    纯文本交互：输入提示，生成响应。
    示例：prompt = "你好，介绍一下自己。"
    """
    # 构建消息（Qwen 格式：list of dicts）
    messages = [{"role": "user", 
                 "content": [{"type": "text", 
                              "text": prompt}]}]
    
    # 应用聊天模板
    text = processor.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    # 处理输入（无图像）
    inputs = processor(text=text, 
                       return_tensors="pt").to(model.device)
    
    # 生成
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # 解码（跳过特殊 token）
    response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 提取 assistant 部分（去除历史）
    response = response.split("assistant\n")[-1].strip() if "assistant\n" in response else response
    return response

In [8]:
print(pure_text_interaction("你好，Qwen3-VL 是什么？"))

你好！Qwen3-VL 是通义实验室推出的一款基于大语言模型的视觉理解模型。它结合了强大的自然语言处理能力与先进的计算机视觉技术，能够理解并生成与图像相关的文本信息，同时也能进行图像生成和描述。Qwen3-VL 旨在为用户提供更加丰富和直观的交互体验，特别是在图像理解和生成方面具有显著优势。


In [10]:
# 2. 核心用法：图文问答（单张图片 + 文本输入）
from PIL import Image
def image_text_qa(image_path: str, prompt: str, max_new_tokens: int = 512) -> str:
    """
    图文问答：输入图片路径和提示，支持描述、视觉 QA、OCR 等。
    示例：
    - 图片描述：prompt = "请详细描述这张图片。"
    - 视觉 QA：prompt = "图片中有什么动物？"
    - OCR：prompt = "提取图片中的所有文字。"
    """
    # 打开图片
    image = Image.open(image_path)
    
    # 构建消息
    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt}
        ]
    }]
    
    # 应用聊天模板
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 处理输入（包括图像）
    inputs = processor(text=text, images=image, return_tensors="pt").to(model.device)
    
    # 生成
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # 解码
    response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 提取 assistant 部分
    response = response.split("assistant\n")[-1].strip() if "assistant\n" in response else response
    return response

In [11]:
print("图片描述：")
print(image_text_qa("../test_data/1.png", "请分析这张图片。"))

图片描述：
好的，这张图片是一张胸部正位X光片，展示了胸廓、肺部、心脏和骨骼的解剖结构。

---

### 图像分析

- **影像类型**：胸部正位X光片（Chest X-ray）
- **观察角度**：前后位（AP view）
- **主要结构**：
  - **骨骼**：清晰可见的锁骨、肩胛骨、肋骨和胸骨。肋骨在肺野中呈清晰的弧形。
  - **胸腔**：包括胸椎、肋骨、胸骨和胸腔内脏器。
  - **肺部**：肺野呈现为双侧的透亮区域，肺纹理清晰，提示肺组织正常。
  - **心脏**：心脏轮廓清晰，位于胸腔中央偏左，大小正常。
  - **膈肌**：膈肌呈穹隆状，位置正常。
  - **纵隔**：纵隔结构清晰，未见明显异常。

- **关键观察点**：
  - **肺部**：双肺透亮度正常，肺野清晰，未见明显积液或气胸。
  - **心脏**：心脏大小正常，轮廓清晰，无明显增大或缩小。
  - **膈肌**：膈肌位置正常，无异常抬高或凹陷。
  - **骨骼**：肋骨、锁骨、肩胛骨等骨骼无明显骨折或畸形。

- **其他信息**：
  - 图片右上角有“L D R C”标记，表明该患者为左侧（L）和右侧（R）的胸部X光片。这通常用于标注患者体位。

---

### 结论

综合以上分析，这张胸部X光片显示了正常胸腔解剖结构。肺部、心脏和骨骼均未见明显异常。因此，该患者目前的胸腔状况是正常的。


In [12]:
print(image_text_qa("../test_data/health-case.png", "提取图片中的所有文字"))

首都儿科研究所附属儿童医院
门诊病历
姓名： 性别：男；年龄： 患者科别：内科门诊；时间：2020-05-01 08:37
主诉：间断气急，运动后出现较多，夜间频繁翻身，无打鼾，有口呼吸，呼吸道感染，1年3-4次，现病史：有AR病史，目前抗过敏治疗，症状有所改善
既往史：体健。
药物过敏史：否认。
个人及家族史：有无精神病史；患儿母亲有荨麻疹病史；有哮喘家族史；否认传染病毒史及家族遗传病史
体格检查：精神可，呼吸平稳，双肺呼吸音清，双肺未闻及啰音。心肺神经系统体查未见异常。
辅助检查：过敏原检测：IgE升高。必要时行心脏、肺部检查
初步诊断：胸闷
过敏性鼻炎[变应性鼻炎]求诊
处
方
名称 规格
孟鲁司特钠咀嚼片 4mg x5片/盒 4mg 口服 每日1次
采乐洛罗贴剂 0.5mg/贴 7贴/盒 1贴 外用 每晚1次
用药天数嘱托 14天
诊断：治疗：不适应诊。
预
约： 科室
号
专业
辅助检查：
打印时间：2020-05-01
处理意见：


In [13]:
# 3. 进阶用法：多轮图文对话（支持多张图片和历史记录）
def multi_turn_image_text_dialogue(history: list, new_image_path: str = None, new_prompt: str = "", max_new_tokens: int = 512) -> tuple[str, list]:
    """
    多轮图文对话：输入历史记录（list of messages），可选新图片和新提示。
    返回：新响应, 更新后的历史。
    
    history 示例（初始为空 list []）：
    history = [
        {"role": "user", "content": [{"type": "image", "image": image1}, {"type": "text", "text": "描述图片。"}]},
        {"role": "assistant", "content": [{"type": "text", "text": "响应1"}]},
        ...
    ]
    
    使用：逐步调用，传入更新后的 history。
    """
    if new_prompt or new_image_path:
        new_content = []
        if new_image_path:
            image = Image.open(new_image_path)
            new_content.append({"type": "image", "image": image})
        if new_prompt:
            new_content.append({"type": "text", "text": new_prompt})
        
        history.append({"role": "user", "content": new_content})
    
    # 应用聊天模板（多轮历史）
    text = processor.apply_chat_template(history, tokenize=False, add_generation_prompt=True)
    
    # 收集所有图像（从历史中提取）
    images = [item["image"] for msg in history for item in msg["content"] if item["type"] == "image"]
    
    # 处理输入
    inputs = processor(text=text, images=images if images else None, return_tensors="pt").to(model.device)
    
    # 生成
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # 解码
    response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 提取新 assistant 响应（去除历史）
    response = response.split("assistant\n")[-1].strip() if "assistant\n" in response else response
    
    # 更新历史：添加 assistant 响应
    history.append({"role": "assistant", "content": [{"type": "text", "text": response}]})
    
    return response, history

In [15]:
# 第一轮：纯文本
history = []
print("\n多轮对话示例：")
resp1, history = multi_turn_image_text_dialogue(history, new_prompt="你好。")
print("Round 1:", resp1)
    
# 第二轮：加图片（假设 'example.jpg'）
resp2, history = multi_turn_image_text_dialogue(history, new_image_path="../test_data/health-case.png", new_prompt="这张图片是什么？")
print("Round 2:", resp2)


多轮对话示例：
Round 1: 你好！有什么我可以帮你的吗？
Round 2: 根据图片内容分析，这是一份**儿童医院的门诊病历**。

具体信息如下：
- **医院**：首都儿科研究所附属儿童医院
- **就诊时间**：2020年05月01日 08:37
- **就诊科室**：呼吸内科门诊
- **患者情况**：男童，因“间歇性气促，运动后出现较多，夜间频繁翻身，无打鼾，有口呼吸，呼吸道感染”就诊
- **初步诊断**：胸闷、过敏性鼻炎（变应性鼻炎）
- **处方**：孟鲁司特钠咀嚼片、利巴韦林贴剂
- **复诊预约**：科室、号别（信息被遮挡）
- **打印时间**：2020年05月01日

这份病历记录了儿童患者因呼吸道感染、过敏性鼻炎等症状就诊的详细信息，包括症状、检查结果、诊断和处方等。
